#### **🧠 OOP — Advanced Concepts (Mixins, MRO, ABC, and Multiple Inheritance Conflicts)**

### What You’ll Learn Here

- Mixins → reuse features without deep inheritance
- MRO (Method Resolution Order) → how Python decides which method to call
- Diamond problem → and how Python solves it
- Abstract Base Classes → enforce design contracts
- Cooperative multiple inheritance using super() -->

🧩 **What is a Mixin?**
- A Mixin is a lightweight class that provides specific functionality to multiple classes.
- It’s not meant to be instantiated — just mixed in to extend other classes.

**Example — LoggingMixin**

In [3]:
# Define the LoggingMixin class
class LoggingMixin:
    # This mixin class provides a simple logging functionality
    def log(self, message):
        # Print the log message with a [LOG] prefix
        print(f"[LOG] {message}")

# Define the FileHandler class that inherits from LoggingMixin
class FileHandler(LoggingMixin):
    # The save method saves data to a file and logs relevant messages
    def save(self, filename, data):
        # Log the message before saving
        self.log(f"Saving to {filename}")
        
        # Open the file and write the data
        with open(filename, 'w') as f:
            f.write(data)
        
        # Log the success message after saving
        self.log("File Saved Successfully")

# Create an instance of FileHandler
fh = FileHandler()

# Use the save method to write data to a file and log the actions
fh.save("test.txt", "Hello Mixins!")


[LOG] Saving to test.txt
[LOG] File Saved Successfully


**Multiple Mixins Example**

In [5]:
# Define a JsonMixin to add JSON serialization capability to a class
class JsonMixin:
    import json  # Importing json inside the mixin class (usually done at the top of the file)
    
    def to_json(self):
        # Convert the object's __dict__ (attributes) to a JSON string
        return self.json.dumps(self.__dict__)

# Define a ReprMixin to add custom string representation to a class
class ReprMixin:
    def __repr__(self):
        # Return a string representation of the object, including class name and its attributes
        return f"{self.__class__.__name__}({self.__dict__})"

# Define the User class, inheriting from both JsonMixin and ReprMixin
class User(JsonMixin, ReprMixin):
    def __init__(self, name, age):
        self.name = name  # Assign the name attribute
        self.age = age    # Assign the age attribute

# Create an instance of User
u = User("Dhiraj", 36)

# Print the object, which will use the __repr__ method from ReprMixin
print(u)  # Output: User({'name': 'Dhiraj', 'age': 36})

# Call the to_json method from JsonMixin to convert the object to a JSON string
print(u.to_json())  # Output: {"name": "Dhiraj", "age": 36}


User({'name': 'Dhiraj', 'age': 36})
{"name": "Dhiraj", "age": 36}


**🧬 MRO — Method Resolution Order**

When multiple inheritance occurs, Python uses MRO to determine which method is called.

In [8]:
# Class A defines the base class with a method 'show' that prints "A"
class A:
    def show(self): 
        print("A")  # Prints 'A' when called

# Class B inherits from class A and overrides the 'show' method
class B(A):
    def show(self):
        print("B")  # Prints 'B' when called (overrides A's show)

# Class C also inherits from class A and overrides the 'show' method
class C(A):
    def show(self):
        print("C")  # Prints 'C' when called (overrides A's show)

# Class D inherits from both B and C, meaning it will inherit from A through both B and C
# In case of method conflicts (like 'show' being defined in B and C), Python will use the MRO (Method Resolution Order) to decide which method to call.
class D(B, C):  
    pass  # No 'show' method is defined in D, so it will use the inherited ones based on MRO

# Create an instance of class D
d = D()

# Calling d.show() will use the MRO to find the 'show' method.
# Since D inherits from B first, Python will use B's version of 'show'.
d.show()  # Output: 'B'

# Print the method resolution order (MRO) for class D
# The MRO is the order in which Python looks for methods in the class hierarchy
# It is important in multiple inheritance situations where methods may be inherited from multiple parent classes.
print(D.__mro__)  
# Output: 
# (<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)


B
(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)


**💥 Diamond Problem & super() Resolution**

The diamond problem happens when multiple inheritance causes ambiguity.

        A
       / \
      B   C
       \ /
        D

In [9]:
# Class A defines the base class with a method 'show' that prints "A"
class A:
    def show(self):
        print("A")  # Prints 'A' when called

# Class B inherits from class A and overrides the 'show' method
# B's show method prints 'B' and then calls the parent (A's) show method using super()
class B(A):
    def show(self):
        print("B")  # Prints 'B' when called
        super().show()  # Calls show() from the parent class (A), printing 'A'

# Class C also inherits from class A and overrides the 'show' method
# C's show method prints 'C' and then calls the parent (A's) show method using super()
class C(A):
    def show(self):
        print("C")  # Prints 'C' when called
        super().show()  # Calls show() from the parent class (A), printing 'A'

# Class D inherits from both B and C. It overrides 'show', prints 'D', and calls 'super()' 
# to invoke the next class's 'show' method based on the Method Resolution Order (MRO)
class D(B, C):
    def show(self):
        print("D")  # Prints 'D' when called
        super().show()  # Calls show() from the next class in the MRO (based on the class order)

# Create an instance of class D and call its show method
# This will follow the MRO and resolve the method calls across multiple classes
D().show()


D
B
C
A


**🔗 Cooperative Multiple Inheritance Pattern**

In [10]:
# Class Base defines the base class with a method 'action' that prints "Base action"
class Base:
    def action(self):
        print("Base action")  # Prints 'Base action' when called

# Class Feature1 inherits from Base and overrides the 'action' method
# It adds extra functionality: printing 'Feature1 start' and 'Feature1 end', and calling Base's action via super()
class Feature1(Base):
    def action(self):
        print("Feature1 start")  # Prints 'Feature1 start'
        super().action()  # Calls the 'action' method of the next class in MRO (Base in this case)
        print("Feature1 end")  # Prints 'Feature1 end' after calling Base's action

# Class Feature2 also inherits from Base and overrides the 'action' method
# It does the same thing as Feature1 but with different print statements
class Feature2(Base):
    def action(self):
        print("Feature2 start")  # Prints 'Feature2 start'
        super().action()  # Calls the 'action' method of the next class in MRO (Base in this case)
        print("Feature2 end")  # Prints 'Feature2 end' after calling Base's action

# Class Combined inherits from both Feature1 and Feature2 (multiple inheritance)
# It overrides the 'action' method and calls the actions in Feature1 and Feature2 via super()
class Combined(Feature1, Feature2):
    def action(self):
        print("Combined start")  # Prints 'Combined start'
        super().action()  # Calls the 'action' method of the next class in MRO (Feature1 in this case)
        print("Combined end")  # Prints 'Combined end' after calling Feature1's action

# Create an instance of Combined and call its action method
Combined().action()


Combined start
Feature1 start
Feature2 start
Base action
Feature2 end
Feature1 end
Combined end


**🧩 Abstract Base Classes (ABCs)**

In [19]:
# Importing the necessary modules
from abc import ABC, abstractmethod  # ABC is used to create an abstract base class, abstractmethod is used to define abstract methods

# Abstract base class for data processors
class DataProcessor(ABC):
    """
    Abstract class that serves as a blueprint for data processors.
    Any specific data processor (e.g., CSV, JSON) must inherit from this class 
    and implement the 'read' and 'process' methods.
    """
    
    @abstractmethod
    def read(self):
        """
        Abstract method to define the reading behavior for the data processor.
        Subclasses must implement this method to handle reading data.
        """
        pass

    @abstractmethod
    def process(self):
        """
        Abstract method to define the data processing behavior.
        Subclasses must implement this method to handle processing the data.
        """
        pass

# A concrete class that processes CSV data
class CSVProcessor(DataProcessor):
    """
    This class implements the abstract methods 'read' and 'process' to handle 
    reading and processing of CSV files. It provides the actual logic 
    for reading a CSV file and processing the data.
    """
    
    def read(self):
        """
        Implements the 'read' method from the DataProcessor class.
        This method simulates reading a CSV file.
        """
        print("📄 Reading CSV file...")  # Placeholder message for simulating reading a CSV file.
    
    def process(self):
        """
        Implements the 'process' method from the DataProcessor class.
        This method simulates processing the data read from a CSV file.
        """
        print("⚙️ Processing CSV data...")  # Placeholder message for simulating data processing.

# Instantiating the CSVProcessor class
dp = CSVProcessor()

# Calling the 'read' method to simulate reading a CSV file
dp.read()

# Calling the 'process' method to simulate processing the data
dp.process()

📄 Reading CSV file...
⚙️ Processing CSV data...


**⚙️ Abstract Properties and Class Methods**

In [23]:
# Importing necessary modules from abc (Abstract Base Class) for abstract classes
from abc import ABC, abstractmethod  # ABC is used to create an abstract base class; abstractmethod defines abstract methods

# Abstract base class representing a geometric shape
class Shape(ABC):
    """
    This is an abstract class for all shapes. It defines the blueprint for any shape, 
    and forces subclasses to implement the 'area' property.
    """
    
    # Abstract property for area, must be implemented by any subclass
    @property
    @abstractmethod
    def area(self):
        """
        Abstract property that should return the area of the shape.
        Subclasses must implement this property to calculate the area of their specific shape.
        """
        pass

# Concrete class representing a Circle, which is a specific shape
class Circle(Shape):
    """
    This class represents a circle, and it inherits from the 'Shape' class. 
    It implements the 'area' property to calculate the area of the circle based on its radius.
    """
    
    def __init__(self, r):
        """
        Constructor to initialize a Circle object with a given radius.
        
        :param r: The radius of the circle (float or int)
        """
        self.r = r  # Assign the radius to an instance variable

    @property
    def area(self):
        """
        Implements the 'area' property defined in the Shape class.
        
        This method calculates and returns the area of the circle using the formula: 
        Area = π * r^2 where r is the radius of the circle.
        """
        return 3.14 * self.r ** 2  # Formula to calculate the area of the circle

# Example of creating a Circle object and getting its area
circle = Circle(5)  # Create a circle with radius 5
print(f"The area of the circle is: {circle.area}")  # Access the 'area' property, which calculates the area


The area of the circle is: 78.5


**🧱 Abstract + Mixin Example (Professional Pattern)**

In [24]:
# Importing the necessary modules for abstract base class functionality and JSON serialization
from abc import ABC, abstractmethod  # ABC is for creating abstract base classes; abstractmethod defines abstract methods
import json  # Module for JSON serialization and deserialization

# Mixin class that provides JSON serialization functionality
class JsonMixin:
    """
    This mixin class provides the functionality to convert an object to a JSON string.
    It adds a `to_json` method that converts the object's __dict__ (attributes) to a JSON string.
    """
    def to_json(self):
        """
        Serializes the object's attributes (stored in the __dict__ attribute) into a JSON string.
        
        :return: JSON string representation of the object's attributes.
        """
        return json.dumps(self.__dict__)  # Convert the object's attributes to JSON string using json.dumps()

# Abstract base class representing storage behavior
class StorageInterface(ABC):
    """
    This is an abstract class that defines the blueprint for storage functionality.
    Any storage class (e.g., FileStorage, DatabaseStorage) must implement the `save` method.
    """
    
    @abstractmethod
    def save(self, data):
        """
        Abstract method for saving data. This must be implemented by subclasses.
        
        :param data: The data to be saved (could be any type of data).
        """
        pass  # Subclasses must implement this method

# Concrete class that implements the storage functionality and uses the JsonMixin for JSON serialization
class FileStorage(StorageInterface, JsonMixin):
    """
    This class implements the `StorageInterface` and mixes in the `JsonMixin` functionality to handle
    storing data in a file after serializing it into JSON format.
    """
    
    def __init__(self, filename):
        """
        Constructor to initialize the file storage with a specified filename.
        
        :param filename: The name of the file where the data will be saved.
        """
        self.filename = filename  # Store the filename where data will be saved

    def save(self, data):
        """
        Implements the `save` method from the `StorageInterface`. This method serializes the provided data 
        to JSON format and writes it to the specified file.
        
        :param data: The data to be saved (usually a dictionary or object).
        """
        with open(self.filename, "w") as f:  # Open the file in write mode
            f.write(self.to_json())  # Write the serialized data (in JSON format) to the file
        print("✅ Data saved to file as JSON.")  # Confirmation message when the data is saved successfully

# Example usage:
# Create an instance of FileStorage with a specific filename
storage = FileStorage("data.json")

# Create some data to store (this can be any object, here it's a simple dictionary)
data = {"name": "John", "age": 30, "city": "New York"}

# Save the data to the file
storage.save(data)


✅ Data saved to file as JSON.



| **Concept**              | **Purpose**                                           | **Keyword**                     |
|--------------------------|-------------------------------------------------------|---------------------------------|
| **Mixin**                | Reuse small features (logging, serialization)        | `multiple inheritance`          |
| **MRO**                  | Controls method lookup order                         | `__mro__`, `super()`            |
| **Diamond Problem**      | Ambiguity resolved by MRO                            | Handled automatically           |
| **Abstract Class**       | Defines template, enforces method existence          | `abc.ABC`, `@abstractmethod`    |
| **Cooperative Inheritance** | Ensures clean chaining                             | `super()` everywhere            |

---

### Explanation:
- **Mixin**: A class that provides a small, reusable piece of functionality, often used via multiple inheritance to add features like logging, authentication, or serialization to other classes.
- **MRO (Method Resolution Order)**: Determines the order in which methods are looked up in the inheritance chain. This helps avoid ambiguity when multiple base classes define the same method.
- **Diamond Problem**: Occurs when a class inherits from two classes that share a common ancestor. This ambiguity is resolved automatically by Python’s MRO mechanism.
- **Abstract Class**: A class that serves as a blueprint for other classes, defining abstract methods that must be implemented in child classes. It cannot be instantiated directly.
- **Cooperative Inheritance**: Ensures that the `super()` function is called properly in classes that use multiple inheritance, allowing methods to be called in the correct order across class hierarchies.


## **🧩 OOP Part 7 — Design Patterns (Factory, Singleton, Strategy, Observer, Adapter)**